In [40]:
import sys
sys.path.append(".")
sys.path.insert(0, ".")
sys.path.append("./neural_correlation")
sys.path.append("./movietag_processing")
import random
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import numpy as np 
import sys
from data import project_dir ,path_to_training_data, path_to_model, patientNums, sr_final_movie_data, patient_features,\
    character_dict, path_to_matlab_generated_movie_data, num_characters, feature_time_width, model_option
from neural_correlation.KnockoutPerformance import KnockoutPerformance, KnockoutKfoldPerformance
from neural_correlation.ModelPerformance import ModelPerformance, KfoldPerformance
from neural_correlation.utilities import load_pickle, dump_pickle
from neural_correlation.MemtestKfoldStats import KfoldStats
from movietag_processing.EpisodeStats import EpisodeStates
import scipy

In [41]:
def rsquared(x, y):
    """ Return R^2 where x and y are array-like."""

    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    return r_value**2

In [42]:
from scipy import stats
def figure4d(path, threshold_num,output_folder, input_folder, mem_test_fn):
    res = []
    patientNums = ['431', '435', '436', '441']

    data = np.array(load_pickle(os.path.join(output_folder,"figure4_one_step.pkl")))
    threshold = 0.5
    threshold_num = threshold_num
    correlation = []
    pval = []
    r_s = []
    for p in patientNums:
        kfoldStats = KfoldStats(p, path, 5, input_folder, mem_test_fn)
        average_stats = kfoldStats.prob_cond_predict_MTL(threshold, threshold_num, clip_mode="clip")
        c, pp = stats.spearmanr(average_stats[0], data)
        #print(p, average_stats[0])
        correlation.append(c)
        pval.append(pp)
        res_v = average_stats[0][np.array([1,2,3,4,6,7,8,9,11,12,13,14])]
        data_v = data[np.array([1,2,3,4,6,7,8,9,11,12,13,14])]
        res.append(res_v)
        r_s.append(rsquared(res_v, data_v))
        
    return res, correlation, pval, r_s

def figure4e(path,threshold_num ,output_folder, input_folder, mem_test_fn):
    res = []
    patientNums = ['431', '435', '436', '441']
    data = np.array(load_pickle(os.path.join(output_folder,"figure4_one_step.pkl")))
    threshold = 0.5
    threshold_num = threshold_num
    correlation = []
    pval = []
    r_s = []
    #print("e")
    for p in patientNums:
        #print(p)
        kfoldStats = KfoldStats(p, path, 5, input_folder, mem_test_fn)
        average_stats = kfoldStats.prob_cond_predict_MTL(threshold, threshold_num, clip_mode="response")
        c, pp = stats.spearmanr(average_stats[0], data)
        #print(p, c)
        correlation.append(c)
        pval.append(pp)
        
        res_v = average_stats[0][np.array([1,2,3,4,6,7,8,9,11,12,13,14])]
        data_v = data[np.array([1,2,3,4,6,7,8,9,11,12,13,14])]
        res.append(res_v)
        r_s.append(rsquared(res_v, data_v))
        
    return res, correlation, pval, r_s
    #savemat(fn,stats)

In [43]:
project_folder = "/media/yipeng/data/movie_2021/Movie_Analysis"
#path = "/media/yipeng/data/movie_2021/Movie_Analysis/paper_results_correct/LSTM_multi_2_KLD"
path = "/media/yipeng/data/movie_2021/Movie_Analysis/CNN_result/LSTM_multi_2_KLD"
#path = "/media/yipeng/data/movie_2021/Movie_Analysis/paper_results/LSTM_multi_2_KLD"
#path = "/media/yipeng/data/movie/data_movie_analysis_final/d"
result_folder = os.path.join(project_folder,"final_result_outputs")
if "LSTM" in path:
    model_option = "LSTM"
    #model_option = "LSTM_MTL"
else:
    model_option = "CNN"
output_folder = os.path.join(result_folder,model_option)

In [44]:
# ######Figure 4#####
from scipy.stats import wilcoxon
#threshold_num = 3
input_folder = os.path.join(project_folder,"final_result_inputs")
import pandas as pd
df = pd.DataFrame()
res_dict = {}
mem_test_fn = "memory_test_prob_noMTL.pkl"
for threshold_num in range(1,5):  
    aa, correlation_a, pval_a, r_a = figure4d(path,threshold_num, output_folder, input_folder, mem_test_fn)
    aa = np.concatenate(aa, 0)
    bb, correlation_b, pval_b, r_b = figure4e(path,threshold_num ,output_folder, input_folder, mem_test_fn)
    bb = np.concatenate(bb, 0)
    sign_rank, pval_r = wilcoxon(aa,bb, alternative="less")
    correlation_a.extend(correlation_b)
    correlation_a.append(sign_rank)
    pval_a.extend(pval_b)
    pval_a.append(pval_r)
    r_a.extend(r_b)
    r_a.append(0)
    key_1 = str(int(threshold_num)) + "_corr"
    key_2 = str(int(threshold_num)) + "_pval"
    key_3 = str(int(threshold_num)) + "_r_square"
    #res_dict = {key_1: correlation_a, key_2:pval_a}
    #df = df.append(res_dict, ignore_index=True)
    df[key_1] = correlation_a
    df[key_2] = pval_a
    df[key_3] = r_a

/home/yipeng/anaconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:160: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/yipeng/anaconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:176: RuntimeWarning: divide by zero encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/home/yipeng/anaconda3/lib/python3.7/site-packages/scipy/stats/_stats_mstats_common.py:183: RuntimeWarning: invalid value encountered in double_scalars
  intercept_stderr = slope_stderr * np.sqrt(ssxm + xmean**2)


In [45]:
df

,1_corr,1_pval,1_r_square,2_corr,2_pval,2_r_square,3_corr,3_pval,3_r_square,4_corr,4_pval,4_r_square
0,0.757622,0.000675,0.000000,0.757622,0.000675,0.000000,0.757622,0.000675,0.000000,0.757622,0.000675,0.000000
1,0.765082,0.000554,0.520411,0.765082,0.000554,0.387491,0.762997,0.000586,0.367318,0.739479,0.001061,0.433980
2,0.828977,0.000072,0.417571,0.885736,0.000005,0.441259,0.851382,0.000029,0.366241,0.799104,0.000204,0.190134
3,0.828977,0.000072,0.368795,0.775206,0.000419,0.305995,0.724421,0.001504,0.486424,0.707025,0.002193,0.349592
4,0.830976,0.000067,0.389680,0.830976,0.000067,0.415552,0.757622,0.000675,0.000000,0.757622,0.000675,0.000000
5,0.788581,0.000283,0.372945,0.775037,0.000421,0.368246,0.740423,0.001037,0.358694,0.740423,0.001037,0.284116
6,0.857356,0.000022,0.366577,0.879761,0.000007,0.417451,0.879761,0.000007,0.525224,0.869306,0.000012,0.538386
7,0.823002,0.000090,0.430068,0.828977,0.000072,0.460285,0.820015,0.000100,0.468336,0.839433,0.000048,0.576062
8,222.000000,0.098389,0.000000,195.000000,0.039853,0.000000,123.000000,0.004188,0.000000,123.000000,0.004188,0.000000


In [46]:
#df.to_csv("MTL_conti.csv")
#df.to_csv("no_MTL_discon_rs.csv")

In [47]:
def activation(path,output_folder, input_folder):
    res = []
    patientNums = ['431', '435', '436', '441']
    data = load_pickle(os.path.join(output_folder,"figure4_one_step.pkl"))
    correlation = []
    pval = []
    #print("e")
    for p in patientNums:
        #print(p)
        kfoldStats = KfoldStats(p, path, 5, input_folder)
        average_stats = kfoldStats.activation()
    

In [48]:
# input_folder = os.path.join(project_folder,"final_result_inputs")
# activation(path, output_folder, input_folder)